In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler drivertorch:  2.4 ; cuda:  2.4.1
detectron2: 0.6

Copyright (c) 2005-2024 NVIDIA Corporation
Built on Tue_Feb_27_16:28:36_Pacific_Standard_Time_2024
Cuda compilation tools, release 12.4, V12.4.99
Build cuda_12.4.r12.4/compiler.33961263_0


In [2]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer

##
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [ ]:
def get_hands_dataset_function(directory, class_labels):
    dataset_dicts = []
    json_files = [f for f in os.listdir(directory) if f.endswith(".json")] # JSON FILES
    for idx, json_file in enumerate(json_files):
        json_path = os.path.join(directory, json_file)
        with open(json_path) as f:
            data = json.load(f)

        record = {}
        img_filename = data["imagePath"]
        img_path = os.path.join(directory, img_filename)
        height, width = data["imageHeight"], data["imageWidth"]

        record["file_name"] = img_path
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
    
        objs = []
        for shape in data["shapes"]:
            label = shape["label"]
            if label not in class_labels:
                continue # Skip unknown labels.

            points = np.array(shape["points"], dtype=np.float32)
            px = points[:, 0]
            py = points[:, 1]

            # Create segmentation polygon
            poly = [(x,y) for x, y in zip(px,py)]
            poly = [p for x in poly for p in x] # Flatten list
            x_min, y_min = np.min(px), np.min(py)
            x_max, y_max = np.max(px), np.max(py)

            obj = {
                "bbox": [float(x_min), float(y_min), float(x_max), float(y_max)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": class_labels.index(label),
            }

            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)

    print(f"Loaded {len(dataset_dicts)} labeled images from {directory}.")
    return dataset_dicts

Registering Datasets

In [7]:
class_labels = ["0", "1", "2", "3", "4", "5"]

DatasetCatalog.register("hands_train", lambda: get_hands_dataset_function("hand_dataset/train/", class_labels))
MetadataCatalog.get("hands_train").set(thing_classes = class_labels)

DatasetCatalog.register("hands_val", lambda: get_hands_dataset_function("hand_dataset/val/", class_labels))
MetadataCatalog.get("hands_val").set(thing_classes = class_labels)

# Metadata just store static info like class names and color maps.
hands_metadata = MetadataCatalog.get("hands_train") 

AssertionError: Dataset 'hands_train' is already registered!

Check if all the data is properly loaded.

In [8]:
dataset_dicts = get_hands_dataset_function("hand_dataset/train/", class_labels)
print(f"Loaded {len(dataset_dicts)} images.")
for d in dataset_dicts:
    print(d["file_name"])

dataset_dicts = get_hands_dataset_function("hand_dataset/val/", class_labels)
print(f"Loaded {len(dataset_dicts)} images.")
for d in dataset_dicts:
    print(d["file_name"])



Loaded 36 labeled images from hand_dataset/train/.
Loaded 36 images.
hand_dataset/train/0_0.jpg
hand_dataset/train/0_1.jpg
hand_dataset/train/0_2.jpg
hand_dataset/train/0_3.jpg
hand_dataset/train/0_4.jpg
hand_dataset/train/0_5.jpg
hand_dataset/train/1_0.jpg
hand_dataset/train/1_1.jpg
hand_dataset/train/1_2.jpg
hand_dataset/train/1_3.jpg
hand_dataset/train/1_4.jpg
hand_dataset/train/1_5.jpg
hand_dataset/train/2_0.jpg
hand_dataset/train/2_1.jpg
hand_dataset/train/2_2.jpg
hand_dataset/train/2_3.jpg
hand_dataset/train/2_4.jpg
hand_dataset/train/2_5.jpg
hand_dataset/train/3_0.jpg
hand_dataset/train/3_1.jpg
hand_dataset/train/3_2.jpg
hand_dataset/train/3_3.jpg
hand_dataset/train/3_4.jpg
hand_dataset/train/3_5.jpg
hand_dataset/train/4_0.jpg
hand_dataset/train/4_1.jpg
hand_dataset/train/4_2.jpg
hand_dataset/train/4_3.jpg
hand_dataset/train/4_4.jpg
hand_dataset/train/4_5.jpg
hand_dataset/train/5_0.jpg
hand_dataset/train/5_1.jpg
hand_dataset/train/5_2.jpg
hand_dataset/train/5_3.jpg
hand_dataset/

Pull up some sample data for visualization

In [9]:
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=hands_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2.imshow("Sample Hands",out.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Setting up Training Configurations

In [10]:
from detectron2.engine import DefaultTrainer
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("hands_train",)
cfg.DATASETS.TEST = ("hands_val",)
cfg.DATALOADER.NUM_WORKERS = 2

# Transfer learning from pretrained COCO Instance segmentation model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 5000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.OUTPUT_DIR = "handsydansy_output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.DEVICE = "cuda"
trainer = DefaultTrainer(cfg) 

[11/10 08:21:29 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Actual Training

In [23]:
from detectron2.data import DatasetCatalog
dataset_dicts = DatasetCatalog.get("hands_train")
print(len(dataset_dicts))
print(dataset_dicts[0])



Loaded 36 labeled images from hand_dataset/train/.
36
{'file_name': 'hand_dataset/train/0_0.jpg', 'image_id': 0, 'height': 550, 'width': 550, 'annotations': [{'bbox': [182.3913116455078, 14.1304349899292, 444.13043212890625, 14.1304349899292], 'bbox_mode': <BoxMode.XYXY_ABS: 0>, 'segmentation': [[196.30435, 542.8261, 210.21739, 388.91306, 185.86957, 244.56522, 182.39131, 147.17392, 222.39131, 52.391304, 265.86957, 14.130435, 352.82608, 17.608696, 383.26086, 53.260868, 397.17392, 51.52174, 444.13043, 120.21739, 422.3913, 227.17392, 371.9565, 315.86957, 348.47827, 401.9565, 349.34784, 544.56525]], 'category_id': 0}]}


In [11]:
trainer.resume_or_load(resume=False)
trainer.train()
# TRAIN

[11/10 08:21:34 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (6, 256, 1, 1) 

[11/10 08:21:35 d2.engine.train_loop]: Starting training from iteration 0


c:\Users\user\.conda\envs\detectry\lib\site-packages\torch\functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3610.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[11/10 08:21:49 d2.utils.events]:  eta: 0:31:03  iter: 19  total_loss: 1.81  loss_cls: 1.808  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.001811  loss_rpn_loc: 0    time: 0.3725  last_time: 0.3761  data_time: 0.3023  last_data_time: 0.0017   lr: 4.9953e-06  max_mem: 1765M
[11/10 08:21:57 d2.utils.events]:  eta: 0:31:38  iter: 39  total_loss: 1.531  loss_cls: 1.528  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.002319  loss_rpn_loc: 0    time: 0.3836  last_time: 0.3701  data_time: 0.0017  last_data_time: 0.0020   lr: 9.9902e-06  max_mem: 1769M
[11/10 08:22:05 d2.utils.events]:  eta: 0:31:31  iter: 59  total_loss: 0.9981  loss_cls: 0.996  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.0005197  loss_rpn_loc: 0    time: 0.3858  last_time: 0.3556  data_time: 0.0017  last_data_time: 0.0017   lr: 1.4985e-05  max_mem: 1769M
[11/10 08:22:13 d2.utils.events]:  eta: 0:31:23  iter: 79  total_loss: 0.4781  loss_cls: 0.4748  loss_box_reg: 0  loss_mask: 0  loss_rpn_cls: 0.002363  loss_rpn_loc: 0 

In [19]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.02   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

[11/10 09:26:48 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from handsydansy_output\model_final.pth ...


c:\Users\user\.conda\envs\detectry\lib\site-packages\fvcore\common\checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.d

Visualization and Result Analysis

In [21]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_hands_dataset_function("hand_dataset/val/", class_labels)
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    print(outputs['instances'])
    v = Visualizer(im[:, :, ::-1],
                   metadata=hands_metadata, 
                   scale=1, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("Hands Validation XDDD",out.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Loaded 12 labeled images from hand_dataset/val/.
Instances(num_instances=0, image_height=550, image_width=550, fields=[pred_boxes: Boxes(tensor([], device='cuda:0', size=(0, 4))), scores: tensor([], device='cuda:0'), pred_classes: tensor([], device='cuda:0', dtype=torch.int64), pred_masks: tensor([], device='cuda:0', size=(0, 550, 550), dtype=torch.bool)])
Instances(num_instances=0, image_height=550, image_width=550, fields=[pred_boxes: Boxes(tensor([], device='cuda:0', size=(0, 4))), scores: tensor([], device='cuda:0'), pred_classes: tensor([], device='cuda:0', dtype=torch.int64), pred_masks: tensor([], device='cuda:0', size=(0, 550, 550), dtype=torch.bool)])
Instances(num_instances=0, image_height=550, image_width=550, fields=[pred_boxes: Boxes(tensor([], device='cuda:0', size=(0, 4))), scores: tensor([], device='cuda:0'), pred_classes: tensor([], device='cuda:0', dtype=torch.int64), pred_masks: tensor([], device='cuda:0', size=(0, 550, 550), dtype=torch.bool)])


In [22]:
print(outputs["instances"].to("cpu"))


Instances(num_instances=0, image_height=550, image_width=550, fields=[pred_boxes: Boxes(tensor([], size=(0, 4))), scores: tensor([]), pred_classes: tensor([], dtype=torch.int64), pred_masks: tensor([], size=(0, 550, 550), dtype=torch.bool)])


In [ ]:
# EVALUATION
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("hands_val", output_dir=os.path.join(cfg.OUTPUT_DIR, "/output_eval"))
val_loader = build_detection_test_loader(cfg, "hands_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`